In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 12.3 MB/s 
     |████████████████████████████████| 163 kB 50.9 MB/s 
     |████████████████████████████████| 7.6 MB 9.0 MB/s 


In [2]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from torchvision import transforms


In [3]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

In [4]:
model_name = "bert-base-uncased"

####bert config just for reference
'''
( vocab_size = 30522hidden_size = 768num_hidden_layers = 12num_attention_heads = 12
intermediate_size = 3072hidden_act = 'gelu'hidden_dropout_prob = 0.1
attention_probs_dropout_prob = 0.1max_position_embeddings = 512
type_vocab_size = 2initializer_range = 0.02layer_norm_eps = 1e-12pad_token_id = 0
position_embedding_type = 'absolute'use_cache = Trueclassifier_dropout = None**kwargs )
'''

max_length = 512

tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

test_size=0.2

dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
documents = dataset.data
labels = dataset.target
(train_texts, valid_texts, train_labels, valid_labels), target_names = train_test_split(documents, labels, test_size=test_size), dataset.target_names
x_train, y_train, x_val, y_val = train_texts, valid_texts, train_labels, valid_labels
# return x_train, y_train, x_val, y_val, target_names

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(y_train, truncation=True, padding=True, max_length=max_length)

In [6]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [7]:
  train_dataset = NewsGroupsDataset(train_encodings, train_labels)
  valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

In [28]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to("cpu")

from sklearn.metrics import accuracy_score

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/5546055f03398095e385d7dc625e636cc8910bf2/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "

In [29]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    #     calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }

In [30]:
def train_model(train_dataset, valid_dataset):
    torch.cuda.empty_cache()
    ##################################Train ARGUMENTS###############################
    training_args = TrainingArguments(
    output_dir='./results',

    ################change number of epochs for testing to 3 , for actual training 1000
    num_train_epochs=1,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=0.0001,
    logging_dir='./logs',
    load_best_model_at_end=True,

    logging_steps=400,
    save_steps=400,
    evaluation_strategy="steps",
    )
    #################################################################################

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        compute_metrics=compute_metrics,
    )

    return trainer

In [31]:

train_model(train_dataset, valid_dataset).train()

using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 15076
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 2513


Step,Training Loss,Validation Loss,Accuracy
400,2.263500,1.460084,0.570822


***** Running Evaluation *****
  Num examples = 3770
  Batch size = 6
Saving model checkpoint to ./results/checkpoint-400
Configuration saved in ./results/checkpoint-400/config.json
Model weights saved in ./results/checkpoint-400/pytorch_model.bin


KeyboardInterrupt: ignored

In [ ]:
train_model(train_dataset, valid_dataset).evaluate()

In [ ]:
model_path = "bert_train"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
torch.save(model.state_dict(), 'bert-model.bin') #can remove if throws error